# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [2]:
from sklearn import ensemble, cross_validation, metrics, tree

import numpy as np

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Генерация и изучение данных

In [3]:
from sklearn.datasets import load_boston
boston = load_boston()
print'data:', boston.data.shape,'\n', 'target:', boston.target.shape

data: (506L, 13L) 
target: (506L,)


In [23]:
print boston

{'data': array([[  6.32000000e-03,   1.80000000e+01,   2.31000000e+00, ...,
          1.53000000e+01,   3.96900000e+02,   4.98000000e+00],
       [  2.73100000e-02,   0.00000000e+00,   7.07000000e+00, ...,
          1.78000000e+01,   3.96900000e+02,   9.14000000e+00],
       [  2.72900000e-02,   0.00000000e+00,   7.07000000e+00, ...,
          1.78000000e+01,   3.92830000e+02,   4.03000000e+00],
       ..., 
       [  6.07600000e-02,   0.00000000e+00,   1.19300000e+01, ...,
          2.10000000e+01,   3.96900000e+02,   5.64000000e+00],
       [  1.09590000e-01,   0.00000000e+00,   1.19300000e+01, ...,
          2.10000000e+01,   3.93450000e+02,   6.48000000e+00],
       [  4.74100000e-02,   0.00000000e+00,   1.19300000e+01, ...,
          2.10000000e+01,   3.96900000e+02,   7.88000000e+00]]), 'feature_names': array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], 
      dtype='|S7'), 'DESCR': "Boston House Prices dataset\n=======

In [4]:
print boston.data.shape

(506L, 13L)


In [5]:
boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], 
      dtype='|S7')

In [6]:
print boston.DESCR

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

### Сплит данных для целей анализа

In [20]:
X_train, X_test = boston.data[:380], boston.data[380:]
y_train, y_test = boston.target[:380], boston.target[380:]

In [21]:
# test
X_train.shape

(380L, 13L)

In [22]:
# функция для записи ответов
def write_answer(ans, ans_number):
    name = 'ans_week4_task2_' + str(ans_number) + '.txt'
    MyAnswer = open(name, "w")
    return MyAnswer.write(str(ans))

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

**L'z(Yi - An-1(xi)) = 2*(An-1(xi) - Yi)** --> **(An-1(xi) - Yi)**

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [10]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [64]:
N = 50
base_algorithms_list = []

DT_Regressor = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
b = np.zeros(len(y_train))

coefficients_list = np.full(N, 0.9)
#print len(gbm_predict(X_train))
DT_Regressor.fit(X_train, y_train - gbm_predict(X_train))
    


DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=42,
           splitter='best')

In [65]:
gbm_predict(X_train)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [67]:
DT_Regressor.predict(X_train)

array([ 26.78      ,  22.75632184,  35.40833333,  35.40833333,
        35.40833333,  32.45      ,  20.4509434 ,  19.5       ,
        19.5       ,  19.5       ,  19.5       ,  20.4509434 ,
        20.4509434 ,  22.75632184,  18.84516129,  22.75632184,
        22.75632184,  18.84516129,  18.84516129,  18.84516129,
        14.67142857,  18.84516129,  14.67142857,  14.67142857,
        14.67142857,  14.67142857,  18.84516129,  14.67142857,
        18.84516129,  18.84516129,  14.67142857,  18.84516129,
        14.67142857,  14.67142857,  14.67142857,  22.75632184,
        22.91666667,  22.75632184,  22.91666667,  30.06315789,
        35.40833333,  30.06315789,  22.75632184,  22.75632184,
        22.75632184,  19.78333333,  19.78333333,  19.5       ,
        19.5       ,  19.5       ,  19.78333333,  22.75632184,
        22.75632184,  22.75632184,  18.84516129,  35.40833333,
        22.75632184,  30.06315789,  22.75632184,  22.75632184,
        20.4509434 ,  20.4509434 ,  22.75632184,  26.78

In [57]:
y_train

array([ 24. ,  21.6,  34.7,  33.4,  36.2,  28.7,  22.9,  27.1,  16.5,
        18.9,  15. ,  18.9,  21.7,  20.4,  18.2,  19.9,  23.1,  17.5,
        20.2,  18.2,  13.6,  19.6,  15.2,  14.5,  15.6,  13.9,  16.6,
        14.8,  18.4,  21. ,  12.7,  14.5,  13.2,  13.1,  13.5,  18.9,
        20. ,  21. ,  24.7,  30.8,  34.9,  26.6,  25.3,  24.7,  21.2,
        19.3,  20. ,  16.6,  14.4,  19.4,  19.7,  20.5,  25. ,  23.4,
        18.9,  35.4,  24.7,  31.6,  23.3,  19.6,  18.7,  16. ,  22.2,
        25. ,  33. ,  23.5,  19.4,  22. ,  17.4,  20.9,  24.2,  21.7,
        22.8,  23.4,  24.1,  21.4,  20. ,  20.8,  21.2,  20.3,  28. ,
        23.9,  24.8,  22.9,  23.9,  26.6,  22.5,  22.2,  23.6,  28.7,
        22.6,  22. ,  22.9,  25. ,  20.6,  28.4,  21.4,  38.7,  43.8,
        33.2,  27.5,  26.5,  18.6,  19.3,  20.1,  19.5,  19.5,  20.4,
        19.8,  19.4,  21.7,  22.8,  18.8,  18.7,  18.5,  18.3,  21.2,
        19.2,  20.4,  19.3,  22. ,  20.3,  20.5,  17.3,  18.8,  21.4,
        15.7,  16.2,

In [55]:
DT_Regressor.predict(X_train)

array([ 26.78      ,  22.75632184,  35.40833333,  35.40833333,
        35.40833333,  32.45      ,  20.4509434 ,  19.5       ,
        19.5       ,  19.5       ,  19.5       ,  20.4509434 ,
        20.4509434 ,  22.75632184,  18.84516129,  22.75632184,
        22.75632184,  18.84516129,  18.84516129,  18.84516129,
        14.67142857,  18.84516129,  14.67142857,  14.67142857,
        14.67142857,  14.67142857,  18.84516129,  14.67142857,
        18.84516129,  18.84516129,  14.67142857,  18.84516129,
        14.67142857,  14.67142857,  14.67142857,  22.75632184,
        22.91666667,  22.75632184,  22.91666667,  30.06315789,
        35.40833333,  30.06315789,  22.75632184,  22.75632184,
        22.75632184,  19.78333333,  19.78333333,  19.5       ,
        19.5       ,  19.5       ,  19.78333333,  22.75632184,
        22.75632184,  22.75632184,  18.84516129,  35.40833333,
        22.75632184,  30.06315789,  22.75632184,  22.75632184,
        20.4509434 ,  20.4509434 ,  22.75632184,  26.78

In [78]:
N = 50
base_algorithms_list = []
#coefficients_list = []

coefficients_list = np.full(N, 0.9)
b = np.zeros(len(y_train))

for i in range(N):
    
    #base_algorithm = DT_Regressor.fit(X_train, y_train - b)
    DT_Regressor = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    algorithm = DT_Regressor.fit(X_train, y_train - gbm_predict(X_train))

    base_algorithms_list.append(algorithm)
    


In [79]:
gbm_predict(X_train)

[23.998263488305913,
 21.598672105576437,
 34.702723092560376,
 33.398493926254766,
 36.202398190643748,
 28.700899560292061,
 22.898551670213784,
 27.097812829034261,
 16.498709045865244,
 18.902345610748899,
 15.000050975975055,
 18.899600562896488,
 21.702142711694393,
 20.402892297439081,
 18.198742394901931,
 19.899386789455608,
 23.096357233109334,
 17.497995437229875,
 20.200235905777461,
 18.195310319998825,
 13.598662847766539,
 19.593395228449584,
 15.204635238598446,
 14.49820159235092,
 15.601273372775292,
 13.900056534704881,
 16.602463457143223,
 14.799558309015977,
 18.395332722039068,
 21.001741966633752,
 12.69812948375874,
 14.502916021357237,
 13.199151495494588,
 13.105789029980755,
 13.49860231035734,
 18.903198385030596,
 20.002482461429771,
 20.99515128762928,
 24.703699459577884,
 30.797139000650184,
 34.900273834120007,
 26.598932322577642,
 25.299008639906464,
 24.7002253940834,
 21.202647681222086,
 19.297852620542752,
 20.000501480613504,
 16.599881437839663

In [76]:
base_algorithms_list[0].predict(X_train)

array([ -2.99389888e+61,  -2.65632112e+61,  -3.97208965e+61,
        -3.97208965e+61,  -3.97208965e+61,  -4.55822046e+61,
        -2.13195305e+61,  -1.65469932e+61,  -1.65469932e+61,
        -2.13195305e+61,  -1.65469932e+61,  -2.13195305e+61,
        -2.13195305e+61,  -2.65632112e+61,  -2.13195305e+61,
        -2.65632112e+61,  -2.65632112e+61,  -2.13195305e+61,
        -2.28891111e+61,  -2.28891111e+61,  -1.65469932e+61,
        -2.13195305e+61,  -2.13195305e+61,  -1.65469932e+61,
        -2.13195305e+61,  -2.28891111e+61,  -2.13195305e+61,
        -2.13195305e+61,  -2.13195305e+61,  -2.13195305e+61,
        -1.65469932e+61,  -2.13195305e+61,  -1.65469932e+61,
        -2.28891111e+61,  -1.65469932e+61,  -2.65632112e+61,
        -2.53118631e+61,  -2.65632112e+61,  -2.53118631e+61,
        -3.25415610e+61,  -3.97208965e+61,  -3.25415610e+61,
        -2.65632112e+61,  -2.65632112e+61,  -2.65632112e+61,
        -2.53118631e+61,  -2.53118631e+61,  -2.53118631e+61,
        -3.33358379e+61,

In [77]:
base_algorithms_list[5].predict(X_train)

array([ -2.99389888e+61,  -2.65632112e+61,  -3.97208965e+61,
        -3.97208965e+61,  -3.97208965e+61,  -4.55822046e+61,
        -2.13195305e+61,  -1.65469932e+61,  -1.65469932e+61,
        -2.13195305e+61,  -1.65469932e+61,  -2.13195305e+61,
        -2.13195305e+61,  -2.65632112e+61,  -2.13195305e+61,
        -2.65632112e+61,  -2.65632112e+61,  -2.13195305e+61,
        -2.28891111e+61,  -2.28891111e+61,  -1.65469932e+61,
        -2.13195305e+61,  -2.13195305e+61,  -1.65469932e+61,
        -2.13195305e+61,  -2.28891111e+61,  -2.13195305e+61,
        -2.13195305e+61,  -2.13195305e+61,  -2.13195305e+61,
        -1.65469932e+61,  -2.13195305e+61,  -1.65469932e+61,
        -2.28891111e+61,  -1.65469932e+61,  -2.65632112e+61,
        -2.53118631e+61,  -2.65632112e+61,  -2.53118631e+61,
        -3.25415610e+61,  -3.97208965e+61,  -3.25415610e+61,
        -2.65632112e+61,  -2.65632112e+61,  -2.65632112e+61,
        -2.53118631e+61,  -2.53118631e+61,  -2.53118631e+61,
        -3.33358379e+61,

In [80]:
y_predict = gbm_predict(X_test)

In [81]:
RMSE = metrics.mean_squared_error(y_predict, y_test) ** 0.5

In [82]:
RMSE

5.4554720745275311

In [83]:
write_answer(RMSE, 2)

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.